In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
import pip
pip.main(['install','seaborn'])
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from keras.applications import DenseNet121, VGG19
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, LeakyReLU, GlobalAveragePooling2D, Conv2D, Input
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
modelclass = 'Male'
suffix = 'Male'
iterations = 5
iterations2 = 5

def CreateModel(learn_rate):
    input_tensor = Input(shape=(256, 256, 3))
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    for layer in base_model.layers[:10]:
        layer.trainable = False
        
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = Dropout(0.5)(x)
    predictions = Dense(5, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learn_rate),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

base_dir = 'Data/'
train_images = np.load(os.path.join(base_dir, 'train_images.npy'))
train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
val_images = np.load(os.path.join(base_dir, 'val_images.npy'))
val_labels = np.load(os.path.join(base_dir, 'val_labels.npy'))

if len(train_images.shape) == 3:
    train_images = np.expand_dims(train_images, axis=-1)
    train_images = np.repeat(train_images, 3, axis=-1)

if len(val_images.shape) == 3:
    val_images = np.expand_dims(val_images, axis=-1)
    val_images = np.repeat(val_images, 3, axis=-1)

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels.flatten())
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

print("Initial class weights:", class_weights)
print("Start time: ", datetime.datetime.now())

best_model_path = f'Models/class{suffix}{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')

best_val_loss = float('inf')
for i in range(iterations):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-7)
    model = CreateModel(1e-5)
    print(f"Training iteration {i+1}/{iterations}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbest.keras')

print("Training Complete")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: seaborn in /opt/conda/lib/python3.11/site-packages (0.13.2)

Requirement already satisfied: numpy!=1.24.0,>=1.20 in /opt/conda/lib/python3.11/site-packages (from seaborn) (1.26.4)

Requirement already satisfied: pandas>=1.2 in /opt/conda/lib/python3.11/site-packages (from seaborn) (2.2.3)

Requirement already satisfied: matplotlib!=3.6.1,>=3.4 in /opt/conda/lib/python3.11/site-packages (from seaborn) (3.9.3)

...

193/194 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - accuracy: 0.3315 - loss: 0.5511

E0000 00:00:1738496146.937742    2072 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738496147.148151    2072 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738496147.987216    2072 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738496148.272217    2072 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.3317 - loss: 0.5509

E0000 00:00:1738496165.185537    2073 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738496165.404116    2073 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738496166.509714    2073 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738496166.803694    2073 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


194/194 ━━━━━━━━━━━━━━━━━━━━ 86s 325ms/step - accuracy: 0.3319 - loss: 0.5507 - val_accuracy: 0.3835 - val_loss: 0.4704 - learning_rate: 1.0000e-05
Epoch 2/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 99s 205ms/step - accuracy: 0.4011 - loss: 0.4852 - val_accuracy: 0.3835 - val_loss: 0.4707 - learning_rate: 1.0000e-05
Epoch 3/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 41s 213ms/step - accuracy: 0.4146 - loss: 0.4858 - val_accuracy: 0.3835 - val_loss: 0.4583 - learning_rate: 1.0000e-05
Epoch 4/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 42s 215ms/step - accuracy: 0.4124 - loss: 0.4661 - val_accuracy: 0.3964 - val_loss: 0.4369 - learning_rate: 1.0000e-05
Epoch 5/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 42s 215ms/step - accuracy: 0.4406 - loss: 0.4445 - val_accuracy: 0.4207 - val_loss: 0.4336 - learning_rate: 1.0000e-05
Epoch 6/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 40s 208ms/step - accuracy: 0.4434 - loss: 0.4339 - val_accuracy: 0.4159 - val_loss: 0.4342 - learning_rate: 1.0000e-05
Epoch 7/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 42s 216ms/step - 

In [2]:
best_model_path = f'Models/class{suffix}secondary{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')
best_val_loss = float('inf')
for i in range (iterations2):
    model = CreateModel(1e-7)
    model.load_weights(f'Models/class_{suffix}{modelclass}_overallbest.keras')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)
    
    print(f"Training iteration {i+1}/{iterations2}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best last val:{val_loss}")
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbestFIN.keras')

/opt/conda/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Training iteration 1/5
Epoch 1/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 50s 230ms/step - accuracy: 0.6932 - loss: 0.2660 - val_accuracy: 0.5518 - val_loss: 0.3961 - learning_rate: 1.0000e-07
Epoch 2/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 41s 209ms/step - accuracy: 0.7014 - loss: 0.2640 - val_accuracy: 0.5453 - val_loss: 0.3972 - learning_rate: 1.0000e-07
Epoch 3/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 41s 210ms/step - accuracy: 0.6797 - loss: 0.2728 - val_accuracy: 0.5485 - val_loss: 0.3962 - learning_rate: 1.0000e-07
Epoch 4/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 41s 210ms/step - accuracy: 0.6874 - loss: 0.2661 - val_accuracy: 0.5485 - val_loss: 0.3963 - learning_rate: 1.0000e-07
Epoch 5/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 41s 210ms/step - accuracy: 0.6946 - loss: 0.2704 - val_accuracy: 0.5485 - val_loss: 0.3963 - learning_rate: 1.0000e-08
Epoch 6/50
194/194 ━━━━━━━━━━━━━━━━━━━━ 41s 210ms/step - accuracy: 0.6898 - loss: 0.2691 - val_accuracy: 0.5485 - val_loss: 0.3964 - learning_rate: 1.0000e-08
Best last val:0.3961098